In [1]:
%matplotlib inline

import csv
import numpy as np
import os
import glob

import data_utils
import matplotlib
import matplotlib.pyplot as plt

%reload_ext autoreload
%autoreload 1
%aimport data_utils


submission_data_path = os.path.join('data', 'test')
train_data_path = os.path.join('data', 'train')


Using TensorFlow backend.


In [ ]:
def load_train_data_from_csv():
    train_files = data_utils.get_train_files(train_data_path)

    train_data_as_array, train_data_as_list = data_utils.get_raw_train_data(train_files)
    
    return train_data_as_array, train_data_as_list

train_data_as_array, train_data_as_list = load_train_data_from_csv()
data_utils.pickle_obj(train_data_as_list, 'train_data_as_list.pkl')

In [ ]:
def load_train_labels_from_csv():
    label_files = data_utils.get_label_files(train_data_path)
    train_labels = [data_utils.load_labels(f) for f in label_files]
    return train_labels

train_labels = load_train_labels_from_csv()
data_utils.pickle_obj(train_labels, 'train_labels.pkl')

In [ ]:
scaler = preprocessing.StandardScaler().fit(train_data_as_array)
data_utils.pickle_obj(scaler, 'scaler.pkl')

## Checkpoint





In [2]:
from sklearn.model_selection import train_test_split

def format_data(scaler, train_data, train_labels, sequence_size):    
    num_train_inputs_total = 0
    num_cols = train_data[0].shape[1]
    num_outputs = train_labels[0].shape[1]
    for td in train_data:
        num_rows = td.shape[0]    
        end_idx = num_rows - (num_rows%sequence_size)
        num_train_inputs_total += end_idx//sequence_size


    train_data_inputs = np.empty((num_train_inputs_total, sequence_size, num_cols))
    train_label_inputs = np.empty((num_train_inputs_total, num_outputs))
    start_idx = 0
    for (td, labels) in zip(train_data, train_labels):
        num_rows = td.shape[0]    
        end_idx = num_rows - (num_rows%sequence_size)
        num_train_inputs = end_idx//sequence_size

        train_data_range = range(start_idx, start_idx + num_train_inputs)
        td = scaler.transform(td)
        train_data_inputs[train_data_range] = \
                np.array([td[i:i+sequence_size] for i in range(0, end_idx, sequence_size)])
        train_label_inputs[train_data_range] = np.array(labels[:num_train_inputs])

        start_idx += num_train_inputs    
    return train_data_inputs, train_label_inputs

def load_training_data(sequence_size):
    train_data = data_utils.unpickle_obj('train_data_as_list.pkl')
    train_labels = data_utils.unpickle_obj('train_labels.pkl')
    scaler = data_utils.unpickle_obj('scaler.pkl')
    X,Y = format_data(scaler, train_data, train_labels, sequence_size)
    return X,Y


def split_train_data(train_data, train_labels, split_fraction=0.1):
    X_train, X_test, Y_train, Y_test = train_test_split(train_data, train_labels, test_size=split_fraction, random_state=42)
    return X_train, X_test, Y_train, Y_test

def get_train_test_data(sequence_size):
    train_data, train_labels = load_training_data(sequence_size)
    X_train, X_test, Y_train, Y_test = split_train_data(train_data, train_labels)
    return X_train, X_test, Y_train, Y_test

sequence_size = 100
X_train, X_test, Y_train, Y_test = get_train_test_data(sequence_size)

In [11]:
from keras.layers.core import Dense
from keras.layers.recurrent import LSTM
from keras.models import Sequential

def build_rnn_model(input_shape, layers):
    model = Sequential()  
    model.add(LSTM(layers[0], return_sequences=True, input_shape=input_shape))
    model.add(LSTM(layers[1], return_sequences=False))
    model.add(Dense(layers[-1]))
    model.compile(loss="mean_squared_error", optimizer="rmsprop")  
    return model

In [18]:
model = build_rnn_model((100,30), (64, 32, 5))
model.fit(X_train, Y_train, batch_size=128, epochs=1, validation_split=0.05)

/home/shaurya/anaconda3/lib/python3.5/site-packages/keras/models.py:837: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 153734 samples, validate on 8092 samples
Epoch 1/1
153734/153734 [==============================] - 210s - loss: 0.0323 - val_loss: 0.0332
